# Part 1: Config

## 1.1 Install Lib

In [1]:
!pip install openai


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install git+https://github.com/openai/whisper.git
!pip install pyannote.audio
!pip install moviepy opencv-python

  Cloning https://github.com/openai/whisper.git to c:\users\nghia\appdata\local\temp\pip-req-build-y16t0h1a
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\Nghia\AppData\Local\Temp\pip-req-build-y16t0h1a'

[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install yt-dlp


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install huggingface_hub[hf_xet]


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install chromedriver-autoinstaller
!apt-get update
!apt-get install -y chromium-browser


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!sudo apt -y update
!sudo apt install -y wget curl unzip
!wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
!dpkg -i libu2f-udev_1.1.4-1_all.deb
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!CHROME_DRIVER_VERSION=$(curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE)
!wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
!unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
!chmod +x /tmp/chromedriver
!mv /tmp/chromedriver /usr/local/bin/chromedriver
!pip install selenium


'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'dpkg' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'dpkg' is not recognized as an internal or external command,
operable program or batch file.
'CHROME_DRIVER_VERSION' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external c


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## 1.2 Import Lib

In [7]:
import os
from dotenv import load_dotenv

In [8]:
import whisper
import cv2
from moviepy.editor import VideoFileClip
from pyannote.audio import Pipeline
from typing import List, Dict
import yt_dlp

In [9]:
import base64
from openai import OpenAI
from pydantic import BaseModel

In [10]:
from moviepy.editor import VideoFileClip

In [11]:
import requests

In [12]:
# For crawling data
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.common.by import By
import concurrent.futures
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import (
    TimeoutException, NoSuchElementException, ElementClickInterceptedException
)

In [13]:
import random
import time

In [14]:
import concurrent.futures

## 1.3 Initialize variables

In [15]:
load_dotenv(dotenv_path="../.env")
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


In [16]:
API_URL = "http://localhost:8000/deepfake/detect/"

In [17]:
# Install Chromedriver
chromedriver_autoinstaller.install()

# Configure Chrome options
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # Run in headless mode
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920x1080')  # Ensure the window size is large enough

chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("accept-language=en-US,en;q=0.9")
chrome_options.add_argument("referer=https://www.google.com/")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")

chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.6478.57 Safari/537.36"
)
# chrome_options.binary_location = '/usr/bin/chromium-browser'

In [18]:
MINIMUM_K = 2

In [19]:
client = OpenAI(
    api_key = OPENAI_API_KEY,
)

# Part 2: Extract statements that need fact-checking

In [20]:
# --- STEP 1: Extract audio from video ---
def extract_audio(video_path: str, audio_dir: str = "audios") -> str:
    if not os.path.exists(audio_dir):
        os.makedirs(audio_dir)
    video = VideoFileClip(video_path)
    audio_path = os.path.basename(video_path).replace('.mp4', '.wav')
    audio_path = os.path.join(audio_dir, audio_path)
    if os.path.exists(audio_path):
        os.remove(audio_path)
    video.audio.write_audiofile(audio_path)
    return audio_path

In [21]:
# --- STEP 2: Diarize audio (identify speakers) ---
def diarize_audio(audio_path: str) -> List[Dict]:
    pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=HUGGINGFACE_TOKEN)
    diarization = pipeline(audio_path)
    speakers = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        speakers.append({
            "start": turn.start,
            "end": turn.end,
            "speaker": speaker
        })
    return speakers


In [22]:
# --- STEP 3: Transcribe audio ---
def transcribe_audio(audio_path: str) -> List[Dict]:
    model = whisper.load_model("base")
    result = model.transcribe(audio_path)
    return result["segments"]


In [23]:
# --- STEP 4: Assign speakers to transcript segments ---
def assign_speakers(segments: List[Dict], speakers: List[Dict]) -> List[Dict]:
    output = []
    for seg in segments:
        speaker_label = "unknown"
        for sp in speakers:
            if sp["start"] <= seg["start"] <= sp["end"]:
                speaker_label = sp["speaker"]
                break
        output.append({
            "start": seg["start"],
            "end": seg["end"],
            "speaker": speaker_label,
            "text": seg["text"].strip()
        })
    return output


In [24]:
# --- STEP 5: Identify speaker names via text cues (OpenAI) ---
class Speaker(BaseModel):
    id: str
    name: str

class ListSpeakers(BaseModel):
    listSpeakers: list[Speaker]


def identify_speaker_names_via_text(transcript: List[Dict]) -> Dict:
    transcript_text = "\n".join(
        [f"{seg['speaker']}: {seg['text']}" for seg in transcript]
    )
    prompt = f"""
    Below is the full transcript of a video, each line contains the speaker (SPEAKER_XX) and the dialogue.

    Analyze to determine if there is any part where the speaker introduces himself or is introduced by someone else.

    Returns a JSON result with the following structure:
    {{
      {{
        id: "SPEAKER_00",
        name: "Name if available",
      }},
      ...
    }}

    If not identified, returns the name field as "Unnamed".

    Transcript:
    {transcript_text}
    """

    response = client.responses.parse(
        model="gpt-4.1-nano",
        input=[
            {"role": "system", "content": "Extract the event information."},
            {
                "role": "user",
                "content": prompt,
            },
        ],
        text_format=ListSpeakers,
    )

    return response.output_parsed



In [25]:
def extract_frames_for_unknown_speakers(
    video_path: str,
    speaker_segments: List[Dict],
    speaker_name_map,  # kiểu: ListSpeakers (đã chứa list[Speaker(id, name)])
    output_dir: str = "frames",
    max_frames_per_speaker: int = 5
):
    import os
    import cv2

    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Tạo dict lookup tên từ speaker_name_map
    speaker_id_to_name = {s.id: s.name for s in speaker_name_map.listSpeakers}
    speaker_frames = {}

    for seg in speaker_segments:
        spk = seg['speaker']
        name = speaker_id_to_name.get(spk, "")
        if name.startswith("Unnamed"):
            # Nếu đã đủ 5 frame thì bỏ qua
            if spk in speaker_frames and len(speaker_frames[spk]) >= max_frames_per_speaker:
                continue

            mid_time = (seg['start'] + seg['end']) / 2
            frame_num = int(mid_time * fps)
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            ret, frame = cap.read()
            if ret:
                frame_path = os.path.join(output_dir, f"{spk}_{int(seg['start'])}.jpg")
                cv2.imwrite(frame_path, frame)
                if spk not in speaker_frames:
                    speaker_frames[spk] = []
                speaker_frames[spk].append({
                    "time": mid_time,
                    "frame_path": frame_path,
                    "text": seg["text"]
                })

    cap.release()
    return speaker_frames


In [26]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def identify_unknown_speakers_with_gpt(speaker_frames: dict) -> dict:
    """
    speaker_frames: {
        "SPEAKER_01": [
            {"time": ..., "frame_path": ..., "text": ...},
            ...
        ],
        ...
    }
    """
    speaker_id_to_name = {}

    for speaker_id, frames in speaker_frames.items():
        print(f"\n🧠 Đang xử lý {speaker_id}...")

        # Chuẩn bị prompt chính
        texts = [f'“{f["text"]}”' for f in frames if f.get("text")]
        combined_text = "\n".join(texts)  # Dùng tối đa 3 đoạn transcript

        prompt = f"""
This is a collection of frames extracted from a video showing one speaker. Based on their appearance and the following quotes, can you identify who they are or make an educated guess?

Quotes:
{combined_text}

Returns only the speaker's name (no further explanation needed).

If you can't tell, reply with "Unnamed".
"""

        # Chuẩn bị ảnh
        content_items = [{"type": "input_text", "text": prompt}]
        for f in frames:
            image_path = f["frame_path"]  # đảm bảo đúng path
            try:
                base64_image = encode_image(image_path)
                content_items.append({
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{base64_image}"
                })
            except Exception as e:
                print(f"❌ Không thể đọc ảnh {image_path}: {e}")

        # Gửi yêu cầu lên GPT
        try:
            response = client.responses.create(
                model="gpt-4.1-mini",
                input=[
                    {
                        "role": "user",
                        "content": content_items
                    }
                ]
            )
            name = response.output_text
            speaker_id_to_name[speaker_id] = name
            print(f"✅ {speaker_id} → {name}")
        except Exception as e:
            print(f"❌ Error for {speaker_id}: {e}")
            speaker_id_to_name[speaker_id] = "Unnamed"

    return speaker_id_to_name


In [27]:
def generate_named_transcript(transcript, speaker_name_map, new_names):
    # Bước 1: Gộp tên từ speaker_name_map và new_names
    speaker_lookup = {}
    for speaker in speaker_name_map.listSpeakers:
        speaker_id = speaker.id
        name = new_names.get(speaker_id, speaker.name)
        speaker_lookup[speaker_id] = name

    # Bước 2: Gán tên rõ ràng vào transcript
    named_transcript = []
    for seg in transcript:
        spk = seg['speaker']
        if spk == "unknown":
            display_name = "Unknown"
        else:
            name = speaker_lookup.get(spk, spk)
            display_name = name if name != "Unnamed" else spk

        named_transcript.append({
            "start": seg["start"],
            "end": seg["end"],
            "speaker": display_name,
            "text": seg["text"]
        })

    # Bước 3: Gộp các đoạn liên tiếp cùng speaker
    if not named_transcript:
        return []

    merged_transcript = []
    current = named_transcript[0]

    for seg in named_transcript[1:]:
        if seg["speaker"] == current["speaker"]:
            # Gộp đoạn
            current["end"] = seg["end"]
            current["text"] += " " + seg["text"]
        else:
            merged_transcript.append(current)
            current = seg

    merged_transcript.append(current)  # Thêm đoạn cuối cùng
    return merged_transcript


In [28]:
class Statement(BaseModel):
    start: float
    end: float
    speaker: str
    text: str
    reason: str  # Tại sao cần kiểm chứng
    context: str

class ListStatement(BaseModel):
    listStatment: List[Statement]


def split_transcript(transcript, chunk_size=100):
    """Chia transcript thành các đoạn nhỏ để tránh quá dài"""
    return [transcript[i:i+chunk_size] for i in range(0, len(transcript), chunk_size)]


def find_checkworthy_statements(final_transcript, model="gpt-4.1-mini"):
    parts = split_transcript(final_transcript, chunk_size=300)
    all_statements = []

    for idx, part in enumerate(parts):
        print(f"🔍 Đang xử lý phần {idx+1}/{len(parts)}...")

        # Tạo văn bản nhập
        lines = [f"[{r['start']:.2f}-{r['end']:.2f}] {r['speaker']}: {r['text']}" for r in part]
        input_text = "\n".join(lines)
        print(input_text)

        prompt = """You are a professional fact-checking assistant.
            Your job is to extract **checkworthy statements** from transcripts of political events, such as debates, interviews, speeches, or hearings.
            Return at **least 2** verifiable statements from 2 different speakers, including unknown speaker (unless there is only 1 speaker, then return 1 statement). If the video is short or has only 1 speaker, extract the entire video.
            
            A **checkworthy statement** typically:
            - Contains a factual claim or statistic.
            - Refers to historical events, wars, or political actions.
            - Makes a cause-effect claim (e.g., "if I were president, this would never happen").
            - Assigns blame or credit for an outcome (e.g., war, economy, healthcare).
            - Makes bold or controversial public assertions.

            Very Important:
            - Only extract **entire continuous speaking segments** from a speaker. That is, if a speaker talks for several sentences in one turn (not interrupted by others), you **must return the full segment verbatim**, not just a partial sentence or fragment.
            - If a speaker talks multiple times (non-consecutively), treat each turn independently — only extract when that turn is checkworthy.

            ### Output Format:
            Return a list of structured statements in this format:
            - `start`: float → start time of the speaker’s turn (in seconds)
            - `end`: float → end time of the speaker’s turn (in seconds)
            - `speaker`: str → name of the speaker
            - `text`: str → full verbatim text spoken by the speaker in that turn
            - `reason`: str → short explanation why this is worth fact-checking
            - `context`: str → provide context including:
                + Where the quote was made (e.g., presidential debate, interview, rally) — infer if not explicit
                + When it happened (date or rough period, e.g., “during the 2024 campaign” or “in June 2025”)
                + The topic under discussion (e.g., foreign policy, war in Ukraine, tax policy)
                + Whether the speaker was replying to someone, and what was asked (if known)
            
            Cannot return null or empty list.
          """

        try:
            response = client.responses.parse(
                model=model,
                input=[
                    {
                        "role": "system",
                        "content": prompt,
                    },
                    {
                        "role": "user",
                        "content": input_text,
                    },
                ],
                text_format=ListStatement,
            )
            statements = response.output_parsed.listStatment
            all_statements.extend(statements)

        except Exception as e:
            print(f"❌ Lỗi ở phần {idx+1}: {e}")

    return all_statements


In [29]:
def extract_frame_for_statement(video_path: str, statement, output_dir="statement_frames"):
    os.makedirs(output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    mid_time = (statement.start + statement.end) / 2
    frame_num = int(mid_time * fps)

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    ret, frame = cap.read()

    if ret:
        filename = f"{statement.speaker}_{int(statement.start*100):06d}.jpg"
        frame_path = os.path.join(output_dir, filename)
        cv2.imwrite(frame_path, frame)
        return frame_path
    else:
        return None

In [30]:
def extract_statements_from_video(video_path: str):
    """
    Extracts the transcript from a video file.
    """
    print("🎬 Extracting audio...")
    audio_path = extract_audio(video_path)
    
    print("🔊 Diarizing speakers...")
    speakers = diarize_audio(audio_path)
    
    print("📝 Transcribing...")
    segments = transcribe_audio(audio_path)
    
    print("👥 Assigning speakers...")
    transcript = assign_speakers(segments, speakers)

    print("🧠 Inferring speaker names from transcript...")
    speaker_name_map = identify_speaker_names_via_text(transcript)
    
    print("🖼️ Extracting frames for unknown speakers...")
    speaker_frames = extract_frames_for_unknown_speakers(video_path, transcript, speaker_name_map)
    
    print("🤖 Identifying unknown speakers with GPT...")
    new_names = identify_unknown_speakers_with_gpt(speaker_frames)

    print("📜 Generating final transcript...")
    final = generate_named_transcript(transcript, speaker_name_map, new_names)
    
    print("🔍 Finding checkworthy statements...")
    final_statements = find_checkworthy_statements(final)
    
    if not final_statements:
        print("❌ No checkworthy statements found.")
        final_statements = find_checkworthy_statements(final)
    
    print(f"✅ Found {len(final_statements)} checkworthy statements.")
    
    final_statements_json = []

    for s in final_statements:
        frame_path = extract_frame_for_statement(video_path, s)

        statement_dict = s.dict()
        statement_dict["frame_path"] = frame_path or "N/A"

        final_statements_json.append(statement_dict)
    
    return final_statements_json
    

# Part 3: Deepfake prediction

In [31]:
def cut_video_into_clips(final_statements_json, video_path: str):
    """
    Cuts the video into clips based on the provided statements.
    Ensures clip end time does not exceed video duration.
    """
    clip_dir = os.path.join("statement_clips", os.path.splitext(os.path.basename(video_path))[0])
    
    # Tạo thư mục hoặc dọn sạch nếu đã tồn tại
    os.makedirs(clip_dir, exist_ok=True)
    for f in os.listdir(clip_dir):
        file_path = os.path.join(clip_dir, f)
        if os.path.isfile(file_path):
            os.remove(file_path)

    # Load video để lấy thời lượng thực tế
    full_video = VideoFileClip(video_path)
    video_duration = full_video.duration
    
    # Kiểm tra nếu final_statements_json chỉ có một phần tử thì lấy toàn bộ video lưu vào clip_dir
    if len(final_statements_json) == 1:
        clip_path = os.path.join(clip_dir, "clip_1.mp4")
        print(f"✂️ Cutting full video to {clip_path}")
        full_video.write_videofile(clip_path, codec="libx264", audio_codec="aac", verbose=False, logger=None)
        full_video.close()
        return clip_dir

    for i, s in enumerate(final_statements_json):
        start, end = float(s['start']), float(s['end'])

        # Bỏ qua nếu start vượt quá thời lượng video
        if start >= video_duration:
            print(f"⚠️ Skip clip_{i+1}: start time {start:.2f}s >= video duration {video_duration:.2f}s")
            continue

        # Giới hạn end không vượt quá video
        end = min(end, video_duration)

        clip_path = os.path.join(clip_dir, f"clip_{i+1}.mp4")
        print(f"✂️ Cutting {clip_path} from {start:.2f}s to {end:.2f}s")

        clip = full_video.subclip(start, end)
        clip.write_videofile(clip_path, codec="libx264", audio_codec="aac", verbose=False, logger=None)

    full_video.close()
    print(f"✅ Finished cutting video into clips. Saved to {clip_dir}")
    return clip_dir


In [32]:
def call_api_detect_deepfake(clip_dir: str):
    """
    Detects deepfake in the video clips.
    """
    # Tạo danh sách file để gửi
    video_files = []
    for filename in os.listdir(clip_dir):
        if filename.endswith(".mp4"):
            file_path = os.path.join(clip_dir, filename)
            video_files.append(("videos", (filename, open(file_path, "rb"), "video/mp4")))
    
    # Gửi yêu cầu POST
    print("📤 Sending batch videos to deepfake API...")
    response = requests.post(API_URL, files=video_files)

    # Kết quả
    if response.status_code == 200:
        result = response.json()
        print("✅ Detection results:")
        for fname, r in result.items():
            print(f"{fname}: {r}")
        return result
    else:
        print(f"❌ Error {response.status_code}: {response.text}")
        
    return None


In [33]:
def detect_deepfake(final_statements_json, video_path: str):
    """
    Main function to detect deepfake in the video.
    """
    print("🎥 Cutting video into clips...")
    
    clip_dir = cut_video_into_clips(final_statements_json, video_path)

    print("🔍 Detecting deepfake in clips...")
    results = call_api_detect_deepfake(clip_dir)

    for idx, statement in enumerate(final_statements_json):
        clip_name = f"clip_{idx+1}.mp4"
        result = results.get(clip_name, {})
        
        # Lấy nhãn deepfake nếu có
        label = result["pred_label"][0]
        pred_score = result["pred"][0]
        
        # Gắn vào statement
        statement["deepfake_label"] = label
        statement["deepfake_score"] = pred_score
    
    print("✅ Finished detecting deepfake.")
    return final_statements_json

# Part 4: Crawl related articles

In [34]:
def search_relevant_links(query):
  driver = webdriver.Chrome(options=chrome_options)

  prompt = f'https://www.bing.com/search?q={query}'
  print(prompt)
  driver.get(prompt)
  time.sleep(random.uniform(1, 10))
  # print(driver.page_source)

  articles = driver.find_elements(By.CSS_SELECTOR, "#b_results li.b_algo")
  link_articles = []
  link_articles.append({
      'title': query[:30],
      'link': prompt,
      # 'summary': summary
  })
  print(f"Found {len(articles)} relevant links:\n{articles}")
  for article in articles[:MINIMUM_K]:  # Giới hạn lấy 5 kết quả đầu tiên
    try:
      title_element = article.find_element(By.TAG_NAME, "h2").find_element(By.TAG_NAME, "a")
      title = title_element.text
      link = title_element.get_attribute('href')
      # summary = article.find_element(By.CLASS_NAME, 'css-16nhkrn').text
      # local = local_query(link)
      link_articles.append({
          'title': title,
          'link': link,
          # 'summary': summary
      })
      print(title)
      print(link)
    except Exception as e:
        print("Lỗi khi trích xuất bài viết:", e)
  driver.quit()

  print(f"Found {len(link_articles)} relevant links:\n{link_articles}")

  return link_articles

In [35]:
def try_dismiss_popups(driver):
    try:
        # Các nút phổ biến cần nhấn
        popup_texts = [
            "Accept Cookies", "Accept All Cookies", "I Accept",
            "Agree", "Press & Hold", "Continue"
        ]
        for text in popup_texts:
            try:
                btn = driver.find_element(
                    By.XPATH,
                    f"//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{text.lower()}')]"
                )
                btn.click()
                print(f"✅ Clicked popup button: '{text}'")
                break
            except NoSuchElementException:
                continue
            except ElementClickInterceptedException:
                continue

        # Tìm các nút có class name chứa 'close'
        close_candidates = driver.find_elements(By.XPATH, "//button[contains(@class, 'close') or contains(translate(@aria-label, 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), 'close')]")

        for btn in close_candidates:
            try:
                btn.click()
                print("✅ Clicked a close button")
                break
            except (ElementClickInterceptedException, Exception):
                continue

    except Exception as e:
        print(f"⚠️ Error while dismissing popup: {e}")

def process_article_link(article, max_retries=5):
    """Hàm xử lý một liên kết riêng lẻ và trả về nội dung gộp các thẻ <p>"""
    article_crawl = {
        "title": article['title'],
        "src": article['link'],
        "contents": ""  # gộp tất cả <p> vào 1 chuỗi
    }

    success = False
    wait_time = 10  # thời gian chờ ban đầu (giây)

    for attempt in range(1, max_retries + 1):
        driver = webdriver.Chrome(options=chrome_options)
        try:
            print(f"⏳ Attempt {attempt}: Crawling {article['link']} with wait_time={wait_time}s")
            driver.get(article['link'])
            time.sleep(wait_time)
            try_dismiss_popups(driver)

            all_elements = driver.find_elements(By.XPATH, ".//p")
            contents = []

            for element in all_elements:
                if element.tag_name == "p":
                    text_content = element.get_attribute("innerText").strip()
                    if text_content:
                        contents.append(text_content)

            article_crawl["contents"] = "\n".join(contents)
            print(f'✅ Crawled content from {article["link"]}:\n{article_crawl["contents"][:500]}...')  # in 500 ký tự đầu tiên
            success = True
            break  # thành công thì thoát

        except Exception as e:
            print(f"⚠️ Attempt {attempt} failed for {article['link']}: {e}")
            wait_time += 300  # tăng thời gian chờ thêm 10s cho mỗi lần thử lại

        finally:
            driver.quit()

    if not success:
        print(f"❌ Failed to crawl article from {article['link']} after {max_retries} attempts.")

    return article_crawl



def crawl_articles(query, crawl_json):
    """Hàm chính để crawl các trang khác"""
    url_articles = search_relevant_links(query)

    # Giới hạn số lượng link cần crawl
    url_articles = url_articles[:MINIMUM_K]

    # Sử dụng Multi-threading để chạy nhiều request song song
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        result = list(executor.map(process_article_link, url_articles))

    # Gộp kết quả vào crawl_json
    crawl_json.extend(result)

In [36]:
def process_single_statement(statement):
    query = statement["text"].strip('"')
    crawl_json = []

    # Crawl theo statement
    crawl_articles(query, crawl_json=crawl_json)

    # Crawl thêm theo context
    context = statement["context"]
    crawl_articles(query=context, crawl_json=crawl_json)

    # Loại bỏ các kết quả None
    crawl_json = [item for item in crawl_json if item is not None]

    # Nối lại toàn bộ nội dung: thêm title và content mỗi bài
    article_texts = "\n\n".join(
        f"### {item.get('title', 'No Title')}\n{item.get('contents', '').strip()}"
        for item in crawl_json
        if item.get("contents")
    )

    # Trả lại enriched statement
    enriched_statement = statement.copy()
    enriched_statement["article_texts"] = article_texts.strip()

    return enriched_statement


def enrich_statements_with_articles(final_statements_json):
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        return list(executor.map(process_single_statement, final_statements_json))


# Part 5: Fact-checking

In [37]:
class FactCheck(BaseModel):
    label: bool  # True = SUPPORTED, False = REFUTED

def fact_check(statement):
    prompt = f"""
You are a professional fact-checking assistant. Your task is to determine whether a specific statement made by a public figure is **factually accurate and verifiably attributed to them**, using only the provided reference documents.

### Instructions:

Follow this strict two-step verification process:

1. **Check if the speaker truly made the statement**:
   - Search the reference documents to confirm whether the speaker is explicitly or clearly quoted as saying this or something semantically equivalent.
   - If you find strong evidence that the speaker made the statement, continue to step 2.
   - If **no such quote or attribution** exists in the documents, set:
     - **label = false**
     - and stop. Do not proceed to step 2.

2. **Evaluate the factual accuracy** of the statement using only the reference materials:
   - If it is directly supported by the documents, set `label = true`.
   - If it is **partially true, misleading, or contradicted**, or if the documents **do not support** it clearly, set `label = false`.

**Important Guidelines:**
- Use only the reference documents. Do not assume or infer information.
- If unsure, default to `label = false`.
- Be skeptical: only statements that are clearly made and clearly true should be labeled `true`.

### Context:
{statement['context']}

### Speaker:
{statement['speaker']}

### Statement:
"{statement['text']}"

### Documents:
{statement['article_texts']}

### Output format:
label: true or false
"""
    try:
        response = client.responses.parse(
            model="gpt-4.1-mini",
            input=[
                {"role": "system", "content": "You are a medical fact-checking expert."},
                {"role": "user", "content": prompt},
            ],
            text_format=FactCheck
        )
        result = response.output_parsed

    except Exception as e:
        print(f"Problem with API: {e}")
        result = "Unverified"

    return result

In [38]:
def fact_check_single_statement(statement):
    print(statement)
    text = statement["text"]

    print(f"🧐 Fact-checking: {text[:80]}...")

    result = fact_check(statement)

    if result == "Unverified":
        statement["label"] = None
        print("❌ Unable to verify statement.")
    else:
        statement["label"] = result.label

    return statement


def run_fact_checks_parallel(enriched_statements, max_workers=4):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(executor.map(fact_check_single_statement, enriched_statements))
    return results


In [39]:
def fact_check_video(video_path: str):
    """
    Main function to fact-check the video.
    """
    print("🎥 Extracting statements from video...")
    final_statements_json = extract_statements_from_video(video_path)
    print(final_statements_json)

    print("🔍 Detecting deepfake...")
    final_statements_json = detect_deepfake(final_statements_json, video_path)
    
    print("✅ Finished detecting deepfake.")
    print(final_statements_json)
    
    for statement in final_statements_json:
        if statement["deepfake_label"] == "FAKE":
            print(f"⚠️ Statement {statement['text']} is marked as deepfake. Skipping fact-check.")
            return {
                "deepfake_label": "FAKE",
                "label": False,
                "statements": final_statements_json
            }


    print("📰 Enriching statements with articles...")
    enriched_statements = enrich_statements_with_articles(final_statements_json)

    print("✅ Finished enriching statements with articles.")

    print("🧪 Running fact-checks on statements...")
    fact_checked_results = run_fact_checks_parallel(enriched_statements)
    fact_checked_results

    for result in fact_checked_results:
        if result["label"] == "False":
            print(f"❌ Statement '{result['text']}' is refuted: {result['explanation']}")
            return {
                "deepfake_label": "REAL",
                "label": False,
                "statements": fact_checked_results
            }


    
    print("✅ All statements are supported or unverified.")
    return {
        "deepfake_label": "REAL",
        "label": True,
        "statements": fact_checked_results
    }
            
            
            

In [40]:
# result = fact_check_video("../data/dfw_youtube_release/1.mp4")

In [41]:
# result

# Part 6: Evaluation

In [42]:
# import json

# INPUT_FILE = "fact_check_results.json"
# OUTPUT_FILE = "fact_check_results_reverified.json"

# # 👇 Cấu hình vùng cần xử lý
# START_INDEX = 0   # chỉnh thành 100, 200... nếu muốn
# NUM_ITEMS = 150    # số lượng video bạn muốn xử lý lại

# # Đọc toàn bộ dữ liệu gốc
# with open(INPUT_FILE, "r", encoding="utf-8") as f:
#     all_results = json.load(f)

# # Cắt ra phần bạn muốn kiểm tra lại
# target_results = all_results[START_INDEX: START_INDEX + NUM_ITEMS]

# # Nếu muốn giữ lại phần trước và sau không thay đổi:
# prefix = all_results[:START_INDEX]
# suffix = all_results[START_INDEX + NUM_ITEMS:]

# reverified_results = []

# for item in target_results:
#     if item.get("label") != True:
#         reverified_results.append(item)
#         continue

#     print(f"🔍 Re-checking statements for {item['video_path']}")

#     statements = item.get("statements", [])
#     if not statements:
#         print("⚠️ No statements found, skipping...")
#         reverified_results.append(item)
#         continue
#     rechecked_statements = run_fact_checks_parallel(statements)

#     all_true = all(st.get("label") is True for st in rechecked_statements)

#     reverified_item = {
#         "video_path": item["video_path"],
#         "deepfake_label": item["deepfake_label"],
#         "label": all_true,
#         "statements": rechecked_statements
#     }

#     reverified_results.append(reverified_item)

# # Gộp lại toàn bộ kết quả
# final_output = prefix + reverified_results + suffix

# # Lưu vào file mới
# with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
#     json.dump(final_output, f, indent=4, ensure_ascii=False)

# print(f"✅ Done. Saved reverified results to {OUTPUT_FILE}")


In [43]:
import json

VIDEO_DIR = "../data/dfw_youtube_release"
SAVE_FILE = "fact_check_results_reverified.json"
BATCH_SIZE = 5  # Mỗi lần lưu kết quả sau 10 video
START_INDEX = 150  # Có thể thay đổi thành 100, 200,...
NUM_VIDEOS = 50  # Số lượng video bạn muốn xử lý lần này

# Lấy danh sách video theo thứ tự số
all_files = sorted([
    f for f in os.listdir(VIDEO_DIR)
    if f.endswith((".mp4", ".avi", ".mov"))
], key=lambda x: int(x.split(".")[0]))  # đảm bảo thứ tự đúng theo số

# Chọn một tập con video cần xử lý
video_files = [
    os.path.join(VIDEO_DIR, f)
    for f in all_files[START_INDEX: START_INDEX + NUM_VIDEOS]
]

# Nếu có file cũ thì load tiếp, không thì tạo danh sách trống
if os.path.exists(SAVE_FILE):
    with open(SAVE_FILE, "r", encoding="utf-8") as f:  # 👈 thêm encoding
        results = json.load(f)
else:
    results = []

# Tránh xử lý lại các video đã có kết quả
processed_videos = {r["video_path"] for r in results}
pending_videos = [vp for vp in video_files if vp not in processed_videos]

batch_counter = 0

for idx, video_path in enumerate(pending_videos, 1):
    print(f"📼 Processing video {idx + START_INDEX}: {video_path}")
    try:
        result = fact_check_video(video_path)
        print(f"✅ Finished processing video {video_path}")
        print(f"Deepfake label: {result['deepfake_label']}, Fact-check label: {result['label']}")
    except Exception as e:
        print(f"⚠️ Error processing video {video_path}: {e}")
        result = None

    results.append({
        "video_path": video_path,
        "deepfake_label": result["deepfake_label"],
        "label": result["label"],
        "statements": result["statements"],
    })

    batch_counter += 1

    # Sau mỗi 10 video thì lưu lại file
    if batch_counter >= BATCH_SIZE:
        with open(SAVE_FILE, "w") as f:
            json.dump(results, f, indent=4)
        print(f"💾 Saved checkpoint after {idx + START_INDEX} videos")
        batch_counter = 0

# Sau vòng lặp, nếu còn batch chưa lưu thì lưu nốt
if batch_counter > 0:
    with open(SAVE_FILE, "w") as f:
        json.dump(results, f, indent=4)
    print(f"💾 Final checkpoint saved after {len(results)} videos.")


📼 Processing video 151: ../data/dfw_youtube_release\150.mp4
🎥 Extracting statements from video...
🎬 Extracting audio...


MoviePy - Writing audio in audios\150.wav


MoviePy - Done.
🔊 Diarizing speakers...


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover
  if ismodule(module) and hasattr(module, '__file__'):



📝 Transcribing...


...

📝 Transcribing...


  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



👥 Assigning speakers...
🧠 Inferring speaker names from transcript...
🖼️ Extracting frames for unknown speakers...
🤖 Identifying unknown speakers with GPT...

🧠 Đang xử lý unknown...
✅ unknown → Unnamed
📜 Generating final transcript...
🔍 Finding checkworthy statements...
🔍 Đang xử lý phần 1/1...
[0.00-2.28] Unknown: Kączy się jeden rok, a zaczyna drugi.
[3.08-54.12] Named: To był dobry rok dla Polski, dla Was, a przede wszystkim dla nas. To będzie trudny rok. Trudny politycznie i gospodarczo jak wiecie. Państwo wiedzą lub jak Państwo nie wiedzą, jeśli oglądają tylko naszą telewizję. W naszych szeregach pojawiają się i będą się pojawiać czarny owce. I tych czarnych owiec jest i będzie coraz więcej. Choćby ten Marjon i jego nazwimy to, może elegancko pokoje na godzinę w poakowskiej kamimce. Może i ci nasi kradną, ale tam ci proszę o tym pamiętać, krali jeszcze bardziej. Może wciskamy Wam kit, ale tam ci wciskali Wam go jeszcze mocniej.
[55.12-68.44] Unknown: Może go im okiem, widać, że w 

  statement_dict = s.dict()



✅ Found 2 checkworthy statements.
[{'start': 3.08, 'end': 54.12, 'speaker': 'Named', 'text': 'To był dobry rok dla Polski, dla Was, a przede wszystkim dla nas. To będzie trudny rok. Trudny politycznie i gospodarczo jak wiecie. Państwo wiedzą lub jak Państwo nie wiedzą, jeśli oglądają tylko naszą telewizję. W naszych szeregach pojawiają się i będą się pojawiać czarny owce. I tych czarnych owiec jest i będzie coraz więcej. Choćby ten Marjon i jego nazwimy to, może elegancko pokoje na godzinę w poakowskiej kamimce. Może i ci nasi kradną, ale tam ci proszę o tym pamiętać, krali jeszcze bardziej. Może wciskamy Wam kit, ale tam ci wciskali Wam go jeszcze mocniej.', 'reason': 'The speaker makes multiple claims about the political and economic conditions in Poland, accuses opponents of more severe corruption and lies, and discusses the current and upcoming political difficulty, which are factual and politically charged claims worth verifying.', 'context': 'This statement was made during a poli